
### PPP Calculation of Csh, Total Sandstone Porosity, Sw in Payzone Using Laminated Aquifer Slope Method


Use this workflow having already computed CshAquifer(array),Rw,a,m,n from the picket plot in excel from the aquifer. Prepare arrays for:
- Gamma Ray values every 2ft of pay
- Nuetron and Density values every 2ft of pay
- Rt values every 2 ft of pay (no need to calculate Rss)

use https://apps.automeris.io/wpd/



In [48]:
import numpy as np
import pandas as pd


#- Gamma Shale: 1 value (local max) **NEAR THE PAY ZONE**
#- Gamma Aquifer: 1 value (local min)
#- Gamma Pay: Gamma ray of the Pay Zone **[Array]**
#- Csh Aquifer: shale concentration of Aquifer 1 value **MUST BE AT SAME DEPTH AS Gamma Aquifer**

#returns [Array]: CshPay: shale concentration along the payzone

def Csh(GammaShale,GammaAquifer,GammaPay,CshAquifer):
    slope = (1-CshAquifer)/(GammaShale-GammaAquifer) #1 value for each well
    CshPay = slope*(GammaPay-GammaShale)+1 #distribution for each 2ft of pay
    return CshPay

#-----------------------------------------------------------------------------------------
#NPay: nuetron porosity every 2 ft of pay [Array]
#DPay: density every 2 ft of pay [Array]
#NShale: nuetron porosity of pure shale (1 value)
#DShale: density porosity of pure shale (1 value)
#CshPay: input the CshPay [Array] from above

#returns [Array]: PhiTotal: corrected, total porosity in the pay zone

def TotalPorosity(NPay,DPay,NShale,DShale,CshPay):
    PhiDcorrected = (DPay-(CshPay*DShale))/(1-CshPay)
    PhiNcorrected = (NPay-(CshPay*NShale))/(1-CshPay)
    PhiTotal = ( ((PhiNcorrected**2)+(PhiDcorrected**2))/2 )**.5
    return PhiTotal
#-----------------------------------------------------------------------------------------

#Rw: single value from picket plot
#Rt: DEEP resistivity every 2 ft of pay [Array]
#phiTotal: input array from above
#a,m,n single values from picket plot:

#returns [2-D Array]: of Sw and Shc

def Saturations(Rw,Rt,phiTotal,a,m,n):
    Sw = ((Rw/Rt)*(a/(phiTotal**m)))**(1/n)
    Shc = 1-Sw
    return np.array([Sw,Shc])
#-----------------------------------------------------------------------------------------

#For loops using arrays: GammaPay,Rw,Npay,Dpay,Rt values. Return distribution, weighted 

data = pd.read_csv('template.csv')
GammaPay = np.array(data['GammaPay'])
Rw = np.array(data['Rw'])
Rt = np.array(data['Rt'])
NPay = np.array(data['Npay'])
DPay = np.array(data['Dpay'])


GammaShale = 105
GammaAquifer = 60
CshAquifer = 0.1667
NShale = .36
DShale = .18
a = 1
m = 2 
n = 2.5


CshPay = np.array([])
for i in range(len(GammaPay)):
    calc = Csh(GammaShale,GammaAquifer,GammaPay[i],CshAquifer)
    CshPay = np.append(CshPay,calc)

PorosityPay = np.array([])
for i in range(len(CshPay)):
    calc1 = TotalPorosity(NPay[i],DPay[i],NShale,DShale,CshPay[i])
    PorosityPay = np.append(PorosityPay,calc1)
    
WaterSaturationPay = np.array([])
OilSaturationPay = np.array([])
for i in range(len(PorosityPay)):
    calc2 = Saturations(Rw[i],Rt[i],PorosityPay[i],a,m,n)
    WaterSaturationPay = np.append(WaterSaturationPay,calc2[0])
    OilSaturationPay = np.append(OilSaturationPay,calc2[1])
    
print(PorosityPay)
print(WaterSaturationPay)

data['PorosityPay'] = PorosityPay
data['WaterSaturationPay'] = WaterSaturationPay

data

    




[0.22442614 0.28392776 0.28696083]
[0.5235171  0.42848567 0.4826407 ]


,Depth,GammaPay,Rw,Rt,Npay,Dpay,Sidewall Porosity,Sidewall Saturation,PorosityPay,WaterSaturationPay
0,7448,83.673469,0.050119,5.017945,0.276868,0.219481,0.25,0.60,0.224426,0.523517
1,7450,72.448980,0.050119,5.173012,0.288127,0.265200,0.25,0.55,0.283928,0.428486
2,7452,82.653061,0.050119,3.760927,0.303207,0.245889,0.26,0.60,0.286961,0.482641
